In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# llamamos la base preprocesada

import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/proyecto/flights_preprocess.csv')

In [16]:
# debido al tamaño de los datos, con el objetivo d eoptimizar recursos, vamos a trabajar con una muestra

data = data.sample(frac=0.1, random_state=42)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55786 entries, 436507 to 4865826
Data columns (total 54 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   FlightDate                               55786 non-null  object 
 1   Airline                                  55786 non-null  int64  
 2   Origin                                   55786 non-null  int64  
 3   Dest                                     55786 non-null  int64  
 4   CRSDepTime                               55786 non-null  int64  
 5   DepTime                                  55786 non-null  int64  
 6   DepDelayMinutes                          55786 non-null  float64
 7   DepDelay                                 55786 non-null  float64
 8   ArrTime                                  55786 non-null  int64  
 9   ArrDelayMinutes                          55786 non-null  float64
 10  AirTime                                  557

In [18]:
# definimos nuestras variables explicativas y objetivo (Arr_Delayed - si los vuelos se retrasaron o no)

X = data.drop(columns=['Arr_Delayed', 'FlightDate'])
y = data['Arr_Delayed']

In [19]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# vamos a reducir la dimensionalidad de nuestras caracteristicas por medio de componentes principales (PCA)

pca = PCA(n_components= 3)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [21]:
# ahora vamos a entrenar distintos modelos. Nuestro baseline model va a ser una regresión logística, mientras que los demás modelos que entrenaremos serán:
# Random Forest, Gradient Boosting, Support Vector Machine y K-Nearest Neighbors

model_log = LogisticRegression(max_iter=1000).fit(X_train_pca, y_train)
model_rf = RandomForestClassifier(random_state=42).fit(X_train_pca, y_train)


In [22]:
model_gb = GradientBoostingClassifier(random_state=42).fit(X_train_pca, y_train)

In [23]:
model_svm = SVC(random_state=42).fit(X_train_pca, y_train)

In [34]:
# definimos una funcion para generar las métricas

def metricas(model):
    y_pred = model.predict(X_test_pca)
    metrics = {
        "Model": "{}".format(model),
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred)
    }
    return metrics

In [35]:
# usemos la funcion con los distintos modelos y generemos una lista

resultados = []
metrica_log = metricas(model_log)
metrica_rf = metricas(model_rf)
metrica_gb = metricas(model_gb)
metrica_svm = metricas(model_svm)
resultados.append(metrica_log)
resultados.append(metrica_rf)
resultados.append(metrica_gb)
resultados.append(metrica_svm)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [36]:
# vamos a comparar los modelos según las distintas metricas:

resultados_df = pd.DataFrame(resultados)

# por accuracy

print("Comparación de Modelos: Accuracy")
print(resultados_df.sort_values(by="Accuracy", ascending=False))

Comparación de Modelos: Accuracy
                                         Model  Accuracy  F1 Score  Precision  \
0            LogisticRegression(max_iter=1000)  0.817351  0.000000   0.000000   
3                         SVC(random_state=42)  0.817351  0.000000   0.000000   
2  GradientBoostingClassifier(random_state=42)  0.817172  0.000000   0.000000   
1      RandomForestClassifier(random_state=42)  0.778455  0.157464   0.257812   

     Recall  
0  0.000000  
3  0.000000  
2  0.000000  
1  0.113346  


In [37]:
# por F1 Score

print("Comparación de Modelos: F1 Score")
print(resultados_df.sort_values(by="F1 Score", ascending=False))

Comparación de Modelos: F1 Score
                                         Model  Accuracy  F1 Score  Precision  \
1      RandomForestClassifier(random_state=42)  0.778455  0.157464   0.257812   
0            LogisticRegression(max_iter=1000)  0.817351  0.000000   0.000000   
2  GradientBoostingClassifier(random_state=42)  0.817172  0.000000   0.000000   
3                         SVC(random_state=42)  0.817351  0.000000   0.000000   

     Recall  
1  0.113346  
0  0.000000  
2  0.000000  
3  0.000000  


In [38]:
# por Precision

print("Comparación de Modelos: Precision")
print(resultados_df.sort_values(by="Precision", ascending=False))

Comparación de Modelos: Precision
                                         Model  Accuracy  F1 Score  Precision  \
1      RandomForestClassifier(random_state=42)  0.778455  0.157464   0.257812   
0            LogisticRegression(max_iter=1000)  0.817351  0.000000   0.000000   
2  GradientBoostingClassifier(random_state=42)  0.817172  0.000000   0.000000   
3                         SVC(random_state=42)  0.817351  0.000000   0.000000   

     Recall  
1  0.113346  
0  0.000000  
2  0.000000  
3  0.000000  


In [33]:
# por Recall

print("Comparación de Modelos: Recall")
print(resultados_df.sort_values(by="Recall", ascending=False))

Comparación de Modelos: Recall
                                               Model  Accuracy  F1 Score  \
1  (DecisionTreeClassifier(max_features='sqrt', r...  0.778455  0.157464   
0                  LogisticRegression(max_iter=1000)  0.817351  0.000000   
2  ([DecisionTreeRegressor(criterion='friedman_ms...  0.817172  0.000000   
3                               SVC(random_state=42)  0.817351  0.000000   

   Precision    Recall  
1   0.257812  0.113346  
0   0.000000  0.000000  
2   0.000000  0.000000  
3   0.000000  0.000000  
